# Genetic Algorithm

We're going to create a list of N numbers that equal X when summed together. If we set N = 5 and X = 200, then these would all be appropriate solutions.

lst = [40,40,40,40,40]

lst = [50,50,50,25,25]

lst = [200,0,0,0,0]

DEAP has the concept of a toolbox, which is a registry of necessary functions. Create a toolbox and register the initialization functions

### Create Chromosome (Genes length, Min, Max)

In [1]:
from random import randint
def individual(length, min, max):
    'create a member of the population'
    return[randint(min,max) for x in range(length)]

In [3]:
'Collection of all individuals is referred to as our Population'

print(individual(5,0,100)) # genes, min, max

[51, 74, 28, 48, 76]


### Create Population (No of Chromosomes, Genes, Min, Max)

In [4]:
def population(count, length, min, max):
    ''' Create a number of individuals
    count: the number of individuals in the population
    length: the number of values per individual
    min: the min possible value in an individual's list of values
    max: the max possible value in an individual's list of values'''
    return [individual(length, min,max) for x in range(count)]


In [5]:
pop=population(4,5,0,100) #no of chromosomes, genes, min, max
pop

[[83, 6, 39, 52, 62],
 [95, 56, 72, 44, 76],
 [29, 51, 2, 49, 90],
 [3, 82, 49, 54, 41]]

Next we need a way to judge the how effective each solution is; to judge the fitness of each individual. Predictably enough, we call this the fitness function. For our problem, we want the fitness to be a function of the distance between the sum of an individuals numbers and the target number X.

### Fitness Function

In [6]:
# Determine the fitness of an individual. Lower is better.
from operator import add
from functools import reduce
def fitness(individual,target):
    '''
    individual: the individual to evaluate
    target: the sum of numbers that individuals are aiming for'''
    sum=reduce(add,individual,0)
    return abs(target-sum)

In [7]:
# Fitness of an Individual Chromosome
x = individual(5,0,100)
print(x)
fitness(x, 200)

[83, 53, 96, 100, 79]


211

In [8]:
# Population Average Fitness
def grade(pop,target):
    summed = reduce(add, (fitness(x, target) for x in pop), 0)
    return summed / (len(pop) * 1.0)

In [9]:
x = population(4,5,0,100) #no of chromosomes, genes, min, max
print(x)
target = 200
grade(x,target)

[[91, 84, 67, 18, 77], [13, 18, 62, 91, 23], [51, 0, 100, 0, 12], [18, 75, 18, 95, 78]]


66.25

### Mutation Function

In [10]:
# Mutation
from random import random, randint
chance_to_mutate = 0.01
for i in pop:
    if chance_to_mutate > random():
        place_to_modify = randint(0,len(i))
        i[place_to_modify] = randint(min(i), max(i))

In [11]:
def evolve(pop, target, retain=0.2, random_select=0.05, mutate=0.01):
    graded = [ (fitness(x, target), x) for x in pop]
    graded = [ x[1] for x in sorted(graded)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]

    # randomly add other individuals to promote genetic diversity
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)
            
    # mutate some individuals
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            # this mutation is not ideal, because it
            # restricts the range of possible values,
            # but the function is unaware of the min/max
            # values used to create the individuals,
            individual[pos_to_mutate] = randint(
                min(individual), max(individual))
    
    # crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = int(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)

    parents.extend(children)
    return parents

In [12]:
target = 371
p_count = 100
i_length = 5
i_min = 0
i_max = 100
p = population(p_count, i_length, i_min, i_max) #no of chromosomes, genes, min, max

fitness_history = [grade(p, target),] #Calculate Population average fitness
for i in range(100):
    p = evolve(p, target)
    fitness_history.append(grade(p, target))

for datum in fitness_history:
    print (datum)

134.57
59.2
20.6
16.23
6.45
2.01
2.03
1.85
1.05
0.05
0.0
0.0
0.0
0.14
0.18
0.0
0.0
1.38
1.92
0.0
0.0
0.0
0.0
0.0
0.0
1.05
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.2
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.74
1.28
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.55
0.93
0.0
0.0
0.0
0.0
0.0
1.45
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.75
1.0
1.02
0.0
0.0
0.0
0.0
0.0
0.66
0.0
1.84
0.0
0.0
0.0
0.0
0.0
1.6
1.2
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0


## Full Code

In [13]:
"""
# Example usage
from genetic import *
target = 371
p_count = 100
i_length = 6
i_min = 0
i_max = 100
p = population(p_count, i_length, i_min, i_max)
fitness_history = [grade(p, target),]
for i in range(100):
    p = evolve(p, target)
    fitness_history.append(grade(p, target))

for datum in fitness_history:
   print datum
"""
from random import randint, random
from operator import add

def individual(length, min, max):
    'Create a member of the population.'
    return [ randint(min,max) for x in range(length) ]

def population(count, length, min, max):
    """
    Create a number of individuals (i.e. a population).

    count: the number of individuals in the population
    length: the number of values per individual
    min: the minimum possible value in an individual's list of values
    max: the maximum possible value in an individual's list of values

    """
    return [ individual(length, min, max) for x in range(count) ]

def fitness(individual, target):
    """
    Determine the fitness of an individual. Higher is better.

    individual: the individual to evaluate
    target: the target number individuals are aiming for
    """
    sum = reduce(add, individual, 0)
    return abs(target-sum)

def grade(pop, target):
    'Find average fitness for a population.'
    summed = reduce(add, (fitness(x, target) for x in pop))
    return summed / (len(pop) * 1.0)

def evolve(pop, target, retain=0.2, random_select=0.05, mutate=0.01):
    graded = [ (fitness(x, target), x) for x in pop]
    graded = [ x[1] for x in sorted(graded)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]
    # randomly add other individuals to
    # promote genetic diversity
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)
    # mutate some individuals
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            # this mutation is not ideal, because it
            # restricts the range of possible values,
            # but the function is unaware of the min/max
            # values used to create the individuals,
            individual[pos_to_mutate] = randint(
                min(individual), max(individual))
    # crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = int(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)        
    parents.extend(children)
    return parents

In [14]:
for i in range(100):
...     p = evolve(p, target)
...     fitness_history.append(grade(p, target))
...
>>> for datum in fitness_history:
...    print (datum)

134.57
59.2
20.6
16.23
6.45
2.01
2.03
1.85
1.05
0.05
0.0
0.0
0.0
0.14
0.18
0.0
0.0
1.38
1.92
0.0
0.0
0.0
0.0
0.0
0.0
1.05
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.2
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.74
1.28
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.55
0.93
0.0
0.0
0.0
0.0
0.0
1.45
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.75
1.0
1.02
0.0
0.0
0.0
0.0
0.0
0.66
0.0
1.84
0.0
0.0
0.0
0.0
0.0
1.6
1.2
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.2
0.0
0.0
0.0
1.28
0.0
0.0
0.0
0.0
0.0
0.0
0.45
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.16
0.85
0.0
0.0
0.0
0.0
0.65
1.87
0.65
0.0
0.0
0.0
1.8
0.0
0.33
0.0
0.0
0.0
0.0
0.0
0.8
0.0
0.0
0.05
0.05
0.02
0.0
0.0
0.24
0.6
0.69
1.61
0.0
0.0
0.0
0.0
0.0
0.0
0.78
0.15
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.82
1.23
0.0
0.0
0.84
0.0
0.0
0.0
0.0
0.0


# Evolve a Neural Network with a Genetic Algorithm

We start by creating a population. This instantiates “count” networks with randomly initialized settings, and adds them to our “pop” list. This is the seed for all generations.

In [14]:
def create_population(self, count):
    """Create a population of random networks.
    Args:
        count (int): Number of networks to generate, aka the
            size of the population
    """
    pop = []
    for _ in range(0, count):
        # Create a random network.
        network = Network(self.nn_param_choices)
        network.create_random()

        # Add the network to our population.
        pop.append(network)

    return pop

Breeding children of our fittest networks is where a lot of the magic happens. Breeding will be different for every application of genetic algorithms, and in our case, we’ve decided to randomly choose parameters for the child from the mother and father.

In [15]:
def breed(self, mother, father):
    """Make two children as parts of their parents.
    Args:
        mother (dict): Network parameters
        father (dict): Network parameters
    """
    children = []
    for _ in range(2):

        child = {}

        # Loop through the parameters and pick params for the kid.
        for param in self.nn_param_choices:
            child[param] = random.choice(
                [mother.network[param], father.network[param]]
            )

        # Now create a network object.
        network = Network(self.nn_param_choices)
        network.create_set(child)

        children.append(network)

    return children

Mutation is also really important as it helps us find chance greatness. In our case, we randomly choose a parameter and then randomly choose a new parameter for it. It can actually end up mutating to the same thing, but that’s all luck of the draw

In [16]:
def mutate(self, network):
    """Randomly mutate one part of the network.
    Args:
        network (dict): The network parameters to mutate
    """
    # Choose a random key.
    mutation = random.choice(list(self.nn_param_choices.keys()))

    # Mutate one of the params.
    network.network[mutation] = random.choice(self.nn_param_choices[mutation])

    return network

The evolve method is where everything is tied together. Each run of this method is a single evolution. Call it enough times, have enough babies and mutations, and… well, evolution!

In [17]:
def evolve(self, pop):
    """Evolve a population of networks.
    Args:
        pop (list): A list of network parameters
    """
    # Get scores for each network.
    graded = [(self.fitness(network), network) for network in pop]

    # Sort on the scores.
    graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]

    # Get the number we want to keep for the next gen.
    retain_length = int(len(graded)*self.retain)

    # The parents are every network we want to keep.
    parents = graded[:retain_length]

    # For those we aren't keeping, randomly keep some anyway.
    for individual in graded[retain_length:]:
        if self.random_select > random.random():
            parents.append(individual)

    # Randomly mutate some of the networks we're keeping.
    for individual in parents:
        if self.mutate_chance > random.random():
            individual = self.mutate(individual)

    # Now find out how many spots we have left to fill.
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []

    # Add children, which are bred from two remaining networks.
    while len(children) < desired_length:

        # Get a random mom and dad.
        male = random.randint(0, parents_length-1)
        female = random.randint(0, parents_length-1)

        # Assuming they aren't the same network...
        if male != female:
            male = parents[male]
            female = parents[female]

            # Breed them.
            babies = self.breed(male, female)

            # Add the children one at a time.
            for baby in babies:
                # Don't grow larger than desired length.
                if len(children) < desired_length:
                    children.append(baby)

    parents.extend(children)

    return parents

# Practice using DEAP : 
Basic Genetic Algorithm

In [15]:
# Here are 5 numbers when added, we get a hundred.
list1 = [100, 0, 0, 0, 0]
list2 = [20, 21, 19, 15, 25]
(sum(list1), sum(list2))

(100, 100)

These 5 numbers in combination make a hundred in sum.

Let’s find the those combination of 5 numbers using GA.

In [25]:
import random
import numpy as np
from deap import algorithms, base, creator, tools

#### Creator
* Meta-factory allowing the run-time creation of classes via both inheritance and composition.
* Attributes, both data and functions, can be dynamically added to existing classes in order to create user-specific new types.
* By using this, the creation of individuals and populations from any data structure ( list, set, dictionary, tree, etc… )

In [26]:
# Creates a new class named "FitnessMin" inheriting from "base.Fitness" with attrebute "weights=(-1.0,)"

# The fitness is a measure of quality of a solution.

creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # -1 -> minimization problem
creator.create("Individual", list, fitness=creator.FitnessMin)

## Toolbox
* Container for the tools (operators) that the user wants to use.
* Manually populated by the user with selected tools.

In [19]:
toolbox = base.Toolbox()
random.randint(0,100) # get a single random number between 0~100
# can be used to create a gene for chromosome

# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 100)

In [23]:
toolbox.attr_bool()

5

In [27]:
# Structure initializers
# Create Individuals and population

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [31]:
print(toolbox.individual())
print(toolbox.population(n=5))

[77, 10, 76, 18, 22]
[[23, 97, 68, 68, 26], [78, 77, 25, 91, 92], [87, 49, 7, 85, 28], [84, 4, 80, 66, 63], [80, 44, 46, 39, 31]]


## The Evaluation Function
Objective function : The function built for our problem

In [32]:
def sum_error(individual):
    return abs(100-sum(individual)),

In [33]:
# smaple individual
ind = toolbox.individual()
print('Random Individula generated is', ind)
print('The error for this individual is:',sum_error(ind))

Random Individula generated is [79, 19, 18, 55, 66]
The error for this individual is: (137,)


## The Genetic Operators
* Operators are just like initializers, except that some are already implemented in the tools module.
* Once you’ve chosen the perfect ones, simply register them in the toolbox.

In [34]:
toolbox.register("evaluate", sum_error)

# Operators for GA
# 1. Cross Over
toolbox.register("mate", tools.cxTwoPoint)

#2. Exploration
toolbox.register("mutate", tools.mutUniformInt, low=0, up=100, indpb=0.2) # Independent probability  : for each attribute to be mutated.# low~up rondom int

# 3. Selection
toolbox.register("select", tools.selTournament, tournsize=3)

### Evolving the Population

#### Creating the Population

In [35]:
pop = toolbox.population(n=100)
for x in pop:
    print (x,end=' ')

[42, 43, 6, 60, 26] [83, 83, 51, 19, 83] [17, 90, 24, 18, 41] [93, 15, 60, 58, 92] [38, 67, 30, 8, 13] [41, 1, 26, 75, 98] [84, 58, 85, 90, 91] [62, 23, 99, 46, 70] [92, 25, 10, 67, 50] [32, 15, 18, 38, 37] [78, 31, 93, 1, 35] [39, 15, 96, 48, 8] [76, 92, 68, 83, 83] [97, 66, 66, 25, 81] [52, 25, 9, 7, 63] [58, 70, 15, 86, 18] [14, 24, 9, 67, 18] [49, 41, 44, 9, 67] [51, 74, 27, 5, 76] [47, 47, 98, 22, 26] [59, 48, 66, 21, 2] [22, 63, 49, 82, 14] [62, 43, 9, 85, 46] [93, 73, 72, 63, 45] [33, 72, 7, 9, 18] [12, 67, 56, 8, 86] [41, 55, 8, 27, 77] [30, 57, 47, 85, 8] [51, 10, 39, 12, 6] [39, 36, 25, 18, 73] [34, 42, 64, 16, 0] [96, 45, 42, 51, 12] [76, 26, 0, 71, 8] [81, 18, 55, 93, 45] [63, 98, 91, 4, 86] [37, 93, 91, 55, 92] [51, 10, 82, 34, 75] [6, 76, 30, 95, 57] [33, 83, 70, 29, 46] [59, 41, 85, 17, 88] [29, 58, 93, 92, 55] [63, 30, 74, 3, 16] [39, 62, 29, 85, 99] [69, 69, 14, 95, 61] [18, 36, 56, 37, 62] [83, 88, 74, 89, 80] [10, 44, 50, 24, 30] [97, 71, 65, 13, 73] [8, 14, 85, 60, 

### The Appeal of Evolution

In [36]:
# Use of a HallOfFame in order to keep track of the best individual to appear in the evolution 
# (it keeps it even in the case it extinguishes)

hof = tools.HallOfFame(1)

In [37]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [31]:
# algorithms : contains useful implements some basic GA

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, 
                               ngen=40, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg   	std    	min	max
0  	100   	146.19	61.2521	0  	287
1  	62    	96.51 	46.878 	0  	212
2  	60    	60.93 	39.9473	0  	191
3  	64    	38.98 	36.008 	0  	189
4  	56    	22.65 	30.6207	0  	175
5  	60    	15.22 	24.4367	0  	142
6  	60    	10.83 	17.9109	0  	102
7  	68    	12.89 	23.5995	0  	112
8  	73    	10.81 	24.3465	0  	140
9  	67    	9.8   	22.3168	0  	120
10 	53    	7.75  	19.59  	0  	109
11 	54    	6.21  	16.6669	0  	104
12 	61    	8.68  	21.4885	0  	103
13 	53    	9.49  	27.1523	0  	181
14 	52    	8.32  	25.3286	0  	137
15 	55    	8.81  	25.7382	0  	134
16 	56    	5.9   	19.2902	0  	99 
17 	57    	8.91  	22.9081	0  	135
18 	58    	4.04  	14.5471	0  	79 
19 	54    	9.15  	25.6111	0  	147
20 	60    	5.6   	17.9883	0  	127
21 	67    	4.49  	16.4058	0  	93 
22 	56    	5.82  	18.3523	0  	140
23 	62    	8.65  	25.2485	0  	146
24 	59    	8.07  	21.525 	0  	145
25 	59    	3.36  	11.2796	0  	92 
26 	58    	5.77  	22.7894	0  	189
27 	63    	10.17 	30.2235	0  	189
28 	69    	8.5

In [38]:
[sum(ind) for ind in pop]

[177,
 319,
 190,
 318,
 156,
 241,
 408,
 300,
 244,
 140,
 238,
 206,
 402,
 335,
 156,
 247,
 132,
 210,
 233,
 240,
 196,
 230,
 245,
 346,
 139,
 229,
 208,
 227,
 118,
 191,
 156,
 246,
 181,
 292,
 342,
 368,
 252,
 264,
 261,
 290,
 327,
 186,
 314,
 308,
 209,
 414,
 158,
 319,
 258,
 281,
 216,
 314,
 269,
 300,
 173,
 244,
 185,
 351,
 151,
 318,
 213,
 295,
 264,
 267,
 290,
 311,
 288,
 177,
 228,
 248,
 267,
 314,
 175,
 218,
 225,
 389,
 270,
 225,
 232,
 240,
 208,
 218,
 251,
 277,
 253,
 159,
 95,
 296,
 240,
 225,
 379,
 151,
 230,
 195,
 207,
 381,
 242,
 263,
 192,
 286]

## Select Top Best

In [39]:
tools.selBest(pop, k=5)

[[42, 43, 6, 60, 26],
 [83, 83, 51, 19, 83],
 [17, 90, 24, 18, 41],
 [93, 15, 60, 58, 92],
 [38, 67, 30, 8, 13]]

# PACKT
# One Max problem
### Generate a Bit pattern of length 75, with 45 ones

In [40]:
import random
from deap import base, creator, tools

In [41]:
# Evaluation function
def eval_func(individual):
    target_sum = 45
    return len(individual) - abs(sum(individual) - target_sum),

'When the number of ones is equal to 45, the return value would be 75.'

'When the number of ones is equal to 45, the return value would be 75.'

The first line creates a single objective maximizing fitness named FitnessMax. The second line deals with producing the individual. In a given process, the first individual that is created is a list of floats. In order to produce this individual, we must create an Individual class using the creator. The fitness attribute will use FitnessMax defined earlier.

In [ ]:
# Create the toolbox with the right parameters
def create_toolbox(num_bits):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    
    
    #A toolbox is an object that is commonly used in DEAP It is used to store 
    #various functions along with their arguments. Let's create this object:
    # Initialize the toolbox
    toolbox = base.Toolbox()
    
    
    
    
    #We will now start registering various functions to this toolbox. 
    # Let's start with the random number generator that generates a random 
    # integer between 0 and 1. This is basically to generate the bit strings:
    

    # Generate attributes 
    toolbox.register("attr_bool", random.randint, 0, 1)
    
    
    
    # Let's register the individual function. The method initRepeat takes three 
    # arguments – a container class for the individual, a function used to fill
    # the container, and the number of times we want the function to repeat itself:

    # Initialize structures
    toolbox.register("individual", tools.initRepeat, creator.Individual, 
        toolbox.attr_bool, num_bits)
    
    
    
    
    # We need to register the population function. We want the population to be 
    # a list of individuals:

    # Define the population to be a list of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    
    
    
    
    #We now need to register the genetic operators. Register the evaluation 
    # function that we defined earlier, which will act as our fitness function.
    # We want the individual, which is a bit pattern, to have 45 ones:

    # Register the evaluation operator 
    toolbox.register("evaluate", eval_func)
    
    
    # Register the crossover operator named mate using the cxTwoPoint method:

    # Register the crossover operator
    toolbox.register("mate", tools.cxTwoPoint)
    
    
    # Register the mutation operator named mutate using mutFlipBit. We need to 
    # specify the probability of each attribute to be mutated using indpb:

    # Register a mutation operator
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    
    
    # Register the selection operator using selTournament. It specifies which 
    # individuals will be selected for breeding:

    # Operator for selecting individuals for breeding
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    return toolbox

In [ ]:
# Define the main function by starting with the length of the bit pattern:

if __name__ == "__main__":
    # Define the number of bits
    num_bits = 75
    
    
    # Create a toolbox using the function we defined earlier:
    # Create a toolbox using the above parameter
    toolbox = create_toolbox(num_bits)


    # We need to seed the random number generator so that we get repeatable results:
    # Seed the random number generator
    random.seed(7)
    
    
    # Create an initial population of, say, 500 individuals using the method 
    # available in the toolbox object. Feel free to change this number and 
    # experiment with it:
    # Create an initial population of 500 individuals
    population = toolbox.population(n=500)

    
    
    # Define the probabilities of crossing and mutating. Again, these are parameters 
    # that are defined by the user. So you can change these parameters and see 
    # how they affect the result:
    # Define probabilities of crossing and mutating
    probab_crossing, probab_mutating  = 0.5, 0.2

    
    
    # Define the number of generations that we need to iterate until the process
    # is terminated. If you increase the number of generations, you are giving 
    # it more freedom to improve the strength of the population:
    
    # Define the number of generations
    num_generations = 60
    
    print('\nStarting the evolution process')
    
    
    
    # Evaluate all the individuals in the population using the fitness functions:
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit
    
    
    # Start iterating through the generations:
    print('\nEvaluated', len(population), 'individuals')
    
    # Iterate through generations
    for g in range(num_generations):
        print("\n===== Generation", g)
        
        
        
        # In each generation, select the next generation individuals using the 
        # selection operator that we registered to the toolbox earlier:
        
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        
        # Clone the selected individuals:
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        
        
        # Apply crossover and mutation on the next generation individuals 
        # using the probability values defined earlier. Once it's done, we need 
        # to reset the fitness values:
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            # Cross two individuals
            if random.random() < probab_crossing:
                toolbox.mate(child1, child2)

                # "Forget" the fitness values of the children
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation to the next generation individuals using the 
        # corresponding probability value that we defined earlier. Once it's 
        # done, reset the fitness value:
        # Apply mutation
        for mutant in offspring:
            # Mutate an individual
            if random.random() < probab_mutating:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with invalid fitness values:
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print('Evaluated', len(invalid_ind), 'individuals')
        
    # Replace the population with the next generation individuals:   
    # The population is entirely replaced by the offspring
        population[:] = offspring
        
       # Print the stats for the current generation to see how it's progressing: 
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in population]
        
        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print('Min =', min(fits), ', Max =', max(fits))
        print('Average =', round(mean, 2), ', Standard deviation =', 
                round(std, 2))
    
    print("\n==== End of evolution")
    
    # Print the final output:
    best_ind = tools.selBest(population, 1)[0]
    print('\nBest individual:\n', best_ind)
    print('\nNumber of ones:', sum(best_ind))

### Full Code

In [ ]:
import random

from deap import base, creator, tools

# Evaluation function
def eval_func(individual):
    target_sum = 45
    return len(individual) - abs(sum(individual) - target_sum),

# Create the toolbox with the right parameters
def create_toolbox(num_bits):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # Initialize the toolbox
    toolbox = base.Toolbox()

    # Generate attributes 
    toolbox.register("attr_bool", random.randint, 0, 1)

    # Initialize structures
    toolbox.register("individual", tools.initRepeat, creator.Individual, 
        toolbox.attr_bool, num_bits)

    # Define the population to be a list of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Register the evaluation operator 
    toolbox.register("evaluate", eval_func)

    # Register the crossover operator
    toolbox.register("mate", tools.cxTwoPoint)

    # Register a mutation operator
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

    # Operator for selecting individuals for breeding
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    return toolbox

if __name__ == "__main__":
    # Define the number of bits
    num_bits = 75

    # Create a toolbox using the above parameter
    toolbox = create_toolbox(num_bits)

    # Seed the random number generator
    random.seed(7)

    # Create an initial population of 500 individuals
    population = toolbox.population(n=500)

    # Define probabilities of crossing and mutating
    probab_crossing, probab_mutating  = 0.5, 0.2

    # Define the number of generations
    num_generations = 60
    
    print('\nStarting the evolution process')
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit
    
    print('\nEvaluated', len(population), 'individuals')
    
    # Iterate through generations
    for g in range(num_generations):
        print("\n===== Generation", g)
        
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            # Cross two individuals
            if random.random() < probab_crossing:
                toolbox.mate(child1, child2)

                # "Forget" the fitness values of the children
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation
        for mutant in offspring:
            # Mutate an individual
            if random.random() < probab_mutating:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print('Evaluated', len(invalid_ind), 'individuals')
        
        # The population is entirely replaced by the offspring
        population[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in population]
        
        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print('Min =', min(fits), ', Max =', max(fits))
        print('Average =', round(mean, 2), ', Standard deviation =', 
                round(std, 2))
    
    print("\n==== End of evolution")
    
    best_ind = tools.selBest(population, 1)[0]
    print('\nBest individual:\n', best_ind)
    print('\nNumber of ones:', sum(best_ind))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from deap import algorithms, base, benchmarks, \
        cma, creator, tools

# Function to create a toolbox
def create_toolbox(strategy):
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("evaluate", benchmarks.rastrigin)

    # Seeed the random number generator
    np.random.seed(7)

    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)

    return toolbox

if __name__ == "__main__":
    # Problem size
    num_individuals = 10
    num_generations = 125

    # Create a strategy using CMA-ES algorithm
    strategy = cma.Strategy(centroid=[5.0]*num_individuals, sigma=5.0, 
            lambda_=20*num_individuals)

    # Create toolbox based on the above strategy
    toolbox = create_toolbox(strategy)

    # Create hall of fame object
    hall_of_fame = tools.HallOfFame(1)

    # Register the relevant stats
    stats = tools.Statistics(lambda x: x.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "std", "min", "avg", "max"
    
    # Objects that will compile the data
    sigma = np.ndarray((num_generations, 1))
    axis_ratio = np.ndarray((num_generations, 1))
    diagD = np.ndarray((num_generations, num_individuals))
    fbest = np.ndarray((num_generations,1))
    best = np.ndarray((num_generations, num_individuals))
    std = np.ndarray((num_generations, num_individuals))

    for gen in range(num_generations):
        # Generate a new population
        population = toolbox.generate()

        # Evaluate the individuals
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
        
        # Update the strategy with the evaluated individuals
        toolbox.update(population)
        
        # Update the hall of fame and the statistics with the
        # currently evaluated population
        hall_of_fame.update(population)
        record = stats.compile(population)
        logbook.record(evals=len(population), gen=gen, **record)
        
        print(logbook.stream)
        
        # Save more data along the evolution 
        sigma[gen] = strategy.sigma
        axis_ratio[gen] = max(strategy.diagD)**2/min(strategy.diagD)**2
        diagD[gen, :num_individuals] = strategy.diagD**2
        fbest[gen] = hall_of_fame[0].fitness.values
        best[gen, :num_individuals] = hall_of_fame[0]
        std[gen, :num_individuals] = np.std(population, axis=0)

    # The x-axis will be the number of evaluations
    x = list(range(0, strategy.lambda_ * num_generations, strategy.lambda_))
    avg, max_, min_ = logbook.select("avg", "max", "min")
    plt.figure()
    plt.semilogy(x, avg, "--b")
    plt.semilogy(x, max_, "--b")
    plt.semilogy(x, min_, "-b")
    plt.semilogy(x, fbest, "-c")
    plt.semilogy(x, sigma, "-g")
    plt.semilogy(x, axis_ratio, "-r")
    plt.grid(True)
    plt.title("blue: f-values, green: sigma, red: axis ratio")

    plt.figure()
    plt.plot(x, best)
    plt.grid(True)
    plt.title("Object Variables")

    plt.figure()
    plt.semilogy(x, diagD)
    plt.grid(True)
    plt.title("Scaling (All Main Axes)")

    plt.figure()
    plt.semilogy(x, std)
    plt.grid(True)
    plt.title("Standard Deviations in All Coordinates")
    
    plt.show()

# Practice 2 : 
Find the Best Tour Route using NSGA-ii

In [ ]:
import pandas as pd
import numpy as np

# 거리계산
from math import radians, cos, sin, asin, sqrt 

# Geo data viz
import folium

In [ ]:
df_spot = pd.read_csv("seoul_street.csv")
df_spot.head(3)

## Viz Data

In [ ]:
m = folium.Map(location=[np.mean(df_spot.lat), np.mean(df_spot.lng)], zoom_start=11, tiles='Stamen Toner')
for index, row in df_spot.iterrows():
    popup_txt = "%s // Score : %s " % (row.street_en, row.score)
    folium.Marker([row.lat, row.lng], popup=popup_txt).add_to(m)
m

## Applying Genetic Algorithm
Setting Things Up

In [ ]:
import random
import numpy as np
from deap import algorithms, base, creator, tools

### Creator

In [ ]:
creator.create("FitnessMulti", base.Fitness, weights=(-1.0,1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

### ToolBox

In [ ]:
toolbox = base.Toolbox()

In [ ]:
# Attribute generator 
toolbox.register("index", np.random.choice, len(df_spot), 5, replace=False) # choose 5 spots

In [ ]:
# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.index)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# sample individual -> just index of the data
ind = toolbox.individual()
ind

In [ ]:
df_spot.ix[ind]

## The Evaluation Function
* objective function 1. total distance -> minimun
* objective function 2. total score -> maximum

In [ ]:
def create_tour(individual):
    return [(df_spot.ix[i].lat, df_spot.ix[i].lng) for i in individual]

In [ ]:
# convert index to geo data for get distance.
tour = create_tour(ind)
tour

In [ ]:
## Function for get a total distance of tour case
def total_distance(tour):
    tour_sum = sum(distance(tour[i], tour[i+1]) for i in range(len(tour)-1))
    return tour_sum

def distance(spot1, spot2):
    # convert decimal degrees to radians 
    lng1, lat1, lng2, lat2 = map(radians, [spot1[0], spot1[1], spot2[0], spot2[1]])
    
    RADIUS = 6371 # FAA approved globe radius in km
    
    dlng = lng2-lng1
    dlat = lat2-lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlng/2)**2
    c = 2 * asin(sqrt(a)) 
    dist = RADIUS * c
    return dist

In [ ]:
# total distance of sample individual
total_distance(tour)

In [ ]:
def plot_tour(ind):
    tour = create_tour(ind)
    m = folium.Map(location=[np.mean(df_spot.lat), np.mean(df_spot.lng)], zoom_start=13)
    path=folium.PolyLine(locations=tour,weight=5)
    m.add_children(path)
    for i,loc in enumerate(ind):
        popup_txt = "%s // Score : %s " % (df_spot.ix[loc].street_en, df_spot.ix[loc].score)
        folium.Marker(tour[i], popup=popup_txt).add_to(m)
    return m

In [ ]:
# vizualize the tour
plot_tour(ind)

In [ ]:
def total_score(individual):
    return sum([df_spot.ix[i]['score'] for i in individual])

In [ ]:
# total scores of the sample individual
total_score(ind)

In [ ]:
def eval_func(individual):
    
    # 1 total distance -> minimun
    t_dist = total_distance(create_tour(individual))
    
    # 2 total score -> maximum
    t_score = total_score(individual)
    
    # 3 penalty
    penalty = len(individual) - len(set(individual))
    t_dist += penalty*1000000
    t_score -= penalty*1000000
    
    return t_dist, t_score

In [ ]:
# total scores of the sample individual
eval_func(ind)

## The Genetic Operators

In [ ]:
toolbox.register("evaluate", eval_func)

In [ ]:
toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", tools.cxTwoPoint)
# tools.mutShuffleIndexes : Shuffle the attributes of the input individual and return the mutant.
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.8)

# Evolving the Population

In [ ]:
POP_SIZE = 100
MAX_GEN = 100
MUT_PROB = 0.2
CX_PROB = 0.8

### Creating the Population

In [ ]:
pop = toolbox.population(n=POP_SIZE)
pop

### The Appeal of Evolution

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0) 
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

In [ ]:
%%time 
result, log = algorithms.eaMuPlusLambda(pop, 
                                     toolbox, 
                                     mu=POP_SIZE, # The number of individuals to select for the next generation.
                                     lambda_= POP_SIZE, # The number of children to produce at each generation.
                                     cxpb= CX_PROB,
                                     mutpb= MUT_PROB, 
                                     stats= stats, 
                                     ngen= MAX_GEN,
                                     verbose= True)

### Make a Decision

In [ ]:
fronts = tools.emo.sortLogNondominated(result, len(result))
fronts

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh import palettes
output_notebook()

In [ ]:
def viz_front(fronts):
    TOOLS = "pan,wheel_zoom,box_zoom,reset,resize"
    hover = HoverTool(
            tooltips=[
                ("index", "$index"),
                ("(x,y)", "($x, $y)"),
                ("individual", "@ind"),
            ]
        )
    front_colors = []
    p = figure(plot_width=700, plot_height=700, tools=[TOOLS,hover], title="NSGAii Test")

    for i,inds in enumerate(fronts):
        par = [(ind, toolbox.evaluate(ind)) for ind in inds]
        source = ColumnDataSource(
                data=dict(
                    x= [p[1][0] for p in par],
                    y= [p[1][1] for p in par],
                    ind= [p[0] for p in par]
                )
            )
        p.circle('x', 'y', size=10, source=source, alpha=0.7, fill_color=palettes.YlGnBu9[i], legend='Front %s'%(i+1), line_color="#ffffff")
    show(p)

In [ ]:
viz_front(fronts)

In [ ]:
fronts[0][0] # One of the Optimal Solution

In [ ]:
df_spot.ix[fronts[0][0]] # Information of that solution

In [ ]:
print(eval_func(fronts[0][99])) # Higher score but Longer distance
plot_tour(fronts[0][99])

In [ ]:
print(eval_func(fronts[0][0]))  # Shorter distance but Lower score
plot_tour(fronts[0][0])

## Create a Bit Pattern of 45 ones

### We want the individual which is a bit pattern to have 45 ones
Generating a bit string with a predefined number of ones

In [35]:
import random

from deap import base, creator, tools
import warnings
warnings.filterwarnings("ignore")

Generate a bit pattern of length 75, with 45 ones

In [36]:
# Evaluation function for target our objective

# Max return value is 75 when number of ones is 45
def eval_func(individual):
    target_sum = 45
    return len(individual) - abs(sum(individual) - target_sum),

In [37]:
# Define a function to create the toolbox with the right parameters




# Define a creator object for fitness function and keep track of individuals
# Fitness class is an abstract class and needs weight attribute to be defined
# First lines create a single objective maximing fitness, named FitnessMax
# Second line deals with creating individuals
# In a given preocess, the first individual that is created is a list of floats
# In order to produce this individual we create an individual class using creator defined in first line
# The fitness attribute will use FitnessMax defined earlier

def create_toolbox(num_bits):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    
    
    
    
    # A toolbox is an object commonly used in DEAP
    # Used to store various functions along with their arguments
    # Create the toolbox Object
    toolbox = base.Toolbox()
    
    
    
    
    
    # We will now start registering various functions to this toolbox
    # Lets start with the random number generator that generates a 
    # random integer between 0 and 1. This is basically to generate bit strings.
    # Generate attributes 
    toolbox.register("attr_bool", random.randint, 0, 1)
    
      
    
    
    # Lets register individual functions
    # The method initrepeat takes 03 arguments: 
    # 1. A container class for the individual (list defined earlier)
    # 2. A function used to fill the container
    # 3. No of times we want the function to repeat itself
    # Initialize structures
    toolbox.register("individual", tools.initRepeat, creator.Individual, 
        toolbox.attr_bool, num_bits)

    
    
    
    # We need to register the population function
    # We want the population to be a list of individuals
    # Define the population to be a list of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    
    
    
    
    # Register the genetic operators
    # Register the evaluation function defined earlier that will act as our
    # fitness function wanting bit pattern to have 45 ones 
    toolbox.register("evaluate", eval_func)

    
    
    # Register the crossover operator named mate using cxTwoPoint method
    toolbox.register("mate", tools.cxTwoPoint)

    
    # Register a mutation operator named mutate using mutFlipBit
    # Specify the probability of each attribute to be mutated using indpb
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    
    
    
    # Register the selection operator using selTournament.
    # It specifies which individuals will be selected for breeding 
    # Operator for selecting individuals for breeding
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    
    # Return the toolbox
    return toolbox 

In [39]:
# Define the main function by starting with the length of bit pattern

if __name__ == "__main__":
    # Define the number of bits
    num_bits = 75
    

    # Create a toolbox using the above parameter
    toolbox = create_toolbox(num_bits)

    # Seed the random number generator to get repeatable results
    random.seed(7)

    # Create an initial population of 500 individuals using the method
    # available in toolbox object
    population = toolbox.population(n=500)

    # Define probabilities of crossing and mutating
    probab_crossing, probab_mutating  = 0.5, 0.2

    # Define the number of generations need to iterate until process is terminated
    # Increasing no of generations means giving it more freedom to improve the strength of population
    num_generations = 60
    
    print('\nStarting the evolution process')
    
    # Evaluate all individuals in the entire population using fitness function with this code
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit
    
    print('\nEvaluated', len(population), 'individuals')
    
    # Iterate through generations
    for g in range(num_generations):
        print("\n===== Generation", g)
        
        
        # In each generation, select next generation individuals using the
        # selection operator that we registered to the toolbox earlier
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        
        
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring using probability 
        # values defined earlier, once its done we need to reset the fitness values
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            # Cross two individuals
            if random.random() < probab_crossing:
                toolbox.mate(child1, child2)

                # "Forget" the fitness values of the children
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation to next generation individuals using corresponding 
        # probability value we defined earlier. Once done reset fitness
        # value
        for mutant in offspring:
            # Mutate an individual
            if random.random() < probab_mutating:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print('Evaluated', len(invalid_ind), 'individuals')
        
        # The population is entirely replaced by the offspring
        population[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in population]
        
        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print('Min =', min(fits), ', Max =', max(fits))
        print('Average =', round(mean, 2), ', Standard deviation =', 
                round(std, 2))
    
    print("\n==== End of evolution")
    
    best_ind = tools.selBest(population, 1)[0]
    print('\nBest individual:\n', best_ind)
    print('\nNumber of ones:', sum(best_ind))


Starting the evolution process

Evaluated 500 individuals

===== Generation 0
Evaluated 297 individuals
Min = 58.0 , Max = 75.0
Average = 70.43 , Standard deviation = 2.91

===== Generation 1
Evaluated 303 individuals
Min = 63.0 , Max = 75.0
Average = 72.44 , Standard deviation = 2.16

===== Generation 2
Evaluated 310 individuals
Min = 65.0 , Max = 75.0
Average = 73.31 , Standard deviation = 1.6

===== Generation 3
Evaluated 273 individuals
Min = 67.0 , Max = 75.0
Average = 73.76 , Standard deviation = 1.41

===== Generation 4
Evaluated 309 individuals
Min = 68.0 , Max = 75.0
Average = 73.87 , Standard deviation = 1.35

===== Generation 5
Evaluated 312 individuals
Min = 68.0 , Max = 75.0
Average = 73.83 , Standard deviation = 1.36

===== Generation 6
Evaluated 308 individuals
Min = 67.0 , Max = 75.0
Average = 73.76 , Standard deviation = 1.5

===== Generation 7
Evaluated 314 individuals
Min = 67.0 , Max = 75.0
Average = 73.85 , Standard deviation = 1.39

===== Generation 8
Evaluated 3

# Simple Regression

In [40]:
import operator
import math
import random
import numpy as np
from deap import algorithms, base, creator, tools, gp

In [42]:
# Create a division operator to handle divide by zero error
# Define new functions
def division_operator(numerator, denominator):
    if denominator == 0:
        return 1

    return numerator / denominator

In [43]:
# Define the evaluation function for fitness calculation
def eval_func(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)

    # Compute the mean squared error
    mse = ((func(x) - (2 * x**3 - 3 * x**2 + 4 * x - 1))**2 for x in points)

    return math.fsum(mse) / len(points),

In [44]:
# Define a Function to create the toolbox
# In order to create toolbox here, we need to create set of primitives
# These premitivies are the operators that will be used during evolution
def create_toolbox():
    pset = gp.PrimitiveSet("MAIN", 1)
    pset.addPrimitive(operator.add, 2)
    pset.addPrimitive(operator.sub, 2)
    pset.addPrimitive(operator.mul, 2)
    pset.addPrimitive(division_operator, 2)
    pset.addPrimitive(operator.neg, 1)
    pset.addPrimitive(math.cos, 1)
    pset.addPrimitive(math.sin, 1)

    pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

    pset.renameArguments(ARG0='x')

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("compile", gp.compile, pset=pset)
    toolbox.register("evaluate", eval_func, points=[x/10. for x in range(-10,10)])
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

    return toolbox

In [45]:
if __name__ == "__main__":
    random.seed(7)

    toolbox = create_toolbox()

    population = toolbox.population(n=450)
    hall_of_fame = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda x: x.fitness.values)
    stats_size = tools.Statistics(len)

    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    # Define parameters
    probab_crossover = 0.4
    probab_mutate = 0.2
    num_generations = 60

    population, log = algorithms.eaSimple(population, toolbox,
            probab_crossover, probab_mutate, num_generations,
            stats=mstats, halloffame=hall_of_fame, verbose=True)


   	      	            fitness            	              size             
   	      	-------------------------------	-------------------------------
gen	nevals	avg    	max    	min    	std    	avg    	max	min	std    
0  	450   	18.6918	47.1923	7.39087	6.27543	3.73556	7  	2  	1.62449
1  	251   	15.4572	41.3823	4.46965	4.54993	3.80222	12 	1  	1.81316
2  	236   	13.2545	37.7223	4.46965	4.06145	3.96889	12 	1  	1.98861
3  	251   	12.2299	60.828 	4.46965	4.70055	4.19556	12 	1  	1.9971 
4  	235   	11.001 	47.1923	4.46965	4.48841	4.84222	13 	1  	2.17245
5  	229   	9.44483	31.478 	4.46965	3.8796 	5.56   	19 	1  	2.43168
6  	225   	8.35975	22.0546	3.02133	3.40547	6.38889	15 	1  	2.40875
7  	237   	7.99309	31.1356	1.81133	4.08463	7.14667	16 	1  	2.57782
8  	224   	7.42611	359.418	1.17558	17.0167	8.33333	19 	1  	3.11127
9  	237   	5.70308	24.1921	1.17558	3.71991	9.64444	23 	1  	3.31365
10 	254   	5.27991	30.4315	1.13301	4.13556	10.5089	25 	1  	3.51898
11 	223   	4.26809	18.7774	0.841562	3.16748	11